<a href="https://colab.research.google.com/github/Meghavarshineks/Speech-emotion-recognition/blob/main/speech_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
root_path = 'gdrive/My Drive/Speech recog/Ravdess/audio_speech_actors_01-24/'

In [1]:
import numpy as np 
import pandas as pd
import librosa
import librosa.display 
import soundfile
import os, glob,sys

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        return result

In [ ]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}


observed_emotions=['calm', 'happy', 'fearful', 'sad','angry']

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    #i=0
    for file in glob.glob(root_path+"Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        #print(file_name,emotion)
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
        #print(feature,emotions)
        #print(i)
        #i=i+1
    #return np.array(x),np.array(y)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:

#X,Y=load_data()
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
df=pd.DataFrame(X)
df['label']=Y
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,label
0,-712.388306,63.240341,-6.698063,12.928182,-9.692841,7.185095,-12.298142,-2.932858,-8.628733,-5.958882,-8.213329,-8.318868,-3.540151,-5.391190,-5.856447,-5.400100,-6.518381,-0.311035,-7.376580,-4.583142,-4.758417,-5.163467,-4.300497,-4.019840,-5.225996,-2.069189,-5.919728,-3.028947,-3.619289,-1.668529,-1.200602,-0.838987,-0.979882,2.792879,1.156514,1.210668,0.850648,1.873484,1.920929,5.635485,...,0.000432,0.000264,0.000327,0.000348,0.000256,0.000199,0.000112,0.000069,0.000049,0.000049,0.000051,0.000041,0.000027,0.000026,0.000041,0.000043,0.000048,0.000035,0.000024,0.000028,0.000019,0.000013,0.000009,0.000007,0.000005,0.000004,0.000002,1.586066e-06,1.076878e-06,8.531343e-07,6.507814e-07,3.157767e-07,2.393200e-07,2.209751e-07,1.868328e-07,1.590003e-07,6.759533e-08,1.000932e-08,7.519310e-09,calm
1,-599.887329,64.439102,-6.381915,10.303803,-6.244401,2.290793,-12.671721,-7.977162,-14.238932,-14.016172,-9.058470,-3.350154,-9.087075,-11.281478,0.246066,-2.009473,-9.165910,-0.663268,-5.202428,-7.709608,-2.851307,-2.003413,-4.028022,-2.931092,-0.774375,1.246649,-1.521887,0.335499,1.098818,1.912535,0.491459,3.205811,2.845409,3.042243,3.754850,2.933762,1.089847,0.870790,-1.424171,-1.535988,...,0.002415,0.001785,0.001842,0.002122,0.002454,0.002326,0.001533,0.001448,0.001170,0.001074,0.000942,0.000509,0.000414,0.000397,0.000407,0.000394,0.000367,0.000308,0.000310,0.000297,0.000263,0.000259,0.000196,0.000153,0.000131,0.000146,0.000090,4.610521e-05,2.757442e-05,2.128181e-05,2.384256e-05,2.641989e-05,2.769627e-05,2.345415e-05,1.330578e-05,8.395259e-06,2.400611e-06,1.909306e-07,1.025534e-08,sad
2,-714.944580,61.081409,-3.209198,6.729263,-4.052040,0.417168,-9.017234,-1.867722,-12.592785,-8.722701,-5.510410,-8.257329,-5.900076,-5.306100,-2.403667,-6.551586,-5.104366,1.111306,-7.570882,-5.997841,-3.218534,-3.474436,-3.770219,-5.410140,-4.613837,-3.946187,-4.912663,-3.068431,-3.998256,-1.903009,-0.331806,-1.932699,-1.248906,0.527815,0.530568,0.129738,0.377466,1.127840,1.971075,3.363005,...,0.000036,0.000034,0.000037,0.000032,0.000034,0.000024,0.000013,0.000009,0.000006,0.000005,0.000003,0.000003,0.000003,0.000004,0.000006,0.000007,0.000010,0.000013,0.000013,0.000008,0.000005,0.000009,0.000006,0.000003,0.000002,0.000002,0.000001,8.747767e-07,4.782062e-07,3.536885e-07,2.087800e-07,1.781192e-07,1.589432e-07,1.324980e-07,9.580431e-08,5.103842e-08,2.411450e-08,7.413765e-09,5.675293e-09,sad
3,-560.943909,59.007679,-21.721348,4.099717,-3.345491,-0.744794,-19.210684,-1.305787,-7.590088,-8.393338,-2.182273,-3.921418,-7.888694,-2.158708,-0.176220,-6.954059,-4.602485,3.350716,-7.865934,-4.916510,-2.450394,-3.382827,-4.303576,-0.807909,-1.455294,-0.107363,-1.840241,3.126824,1.385932,2.728318,2.840037,1.761733,0.383645,1.888118,2.619172,2.096194,5.148524,5.815701,3.486905,2.302528,...,0.000360,0.000374,0.000408,0.000405,0.000266,0.000247,0.000241,0.000162,0.000195,0.000212,0.000175,0.000134,0.000136,0.000145,0.000192,0.000242,0.000223,0.000234,0.000215,0.000211,0.000125,0.000111,0.000074,0.000054,0.000042,0.000028,0.000020,1.370840e-05,9.621894e-06,1.033495e-05,7.868209e-06,6.618394e-06,5.796551e-06,3.801436e-06,2.812426e-06,2.378993e-06,9.983867e-07,8.456907e-08,8.641630e-09,happy
4,-549.236206,55.645756,-18.499104,3.886537,2.170367,-1.320795,-13.935467,-1.307494,-6.164294,-6.724278,-1.748288,-2.759629,-6.336930,-2.210993,3.369204,-3.728796,-4.467932,2.997232,-4.070324,-4.058586,-1.952857,-2.550913,-3.447933,-1.292407,-1.157766,-1.425361,-2.301708,0.266337,0.086388,-0.239441,0.076293,1.605086,1.152786,3.166218,2.452166,-0.048638,2.362867,3.411358,4.714571,6.882650,...,0.000423,0.000492,0.000452,0.000297,0.000519,0.000333,0.000296,0.000229,0.000229,0.000251,0.000127,0.000074,0.000065,0.0

In [ ]:
df.label.value_counts()

fearful    199
angry      196
sad        192
happy      191
calm       190
Name: label, dtype: int64

In [ ]:
def score_to_numeric(x):
    if x=='calm':
        return 0
    if x=='happy':
        return 1
    if x=='fearful':
        return 2
    if x=='disgust':
        return 3

In [ ]:
df['num_label'] = df['label'].apply(score_to_numeric)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,label,num_label
0,-712.388306,63.240341,-6.698063,12.928182,-9.692841,7.185095,-12.298142,-2.932858,-8.628733,-5.958882,-8.213329,-8.318868,-3.540151,-5.391190,-5.856447,-5.400100,-6.518381,-0.311035,-7.376580,-4.583142,-4.758417,-5.163467,-4.300497,-4.019840,-5.225996,-2.069189,-5.919728,-3.028947,-3.619289,-1.668529,-1.200602,-0.838987,-0.979882,2.792879,1.156514,1.210668,0.850648,1.873484,1.920929,5.635485,...,0.000264,0.000327,0.000348,0.000256,0.000199,0.000112,0.000069,0.000049,0.000049,0.000051,0.000041,0.000027,0.000026,0.000041,0.000043,0.000048,0.000035,0.000024,0.000028,0.000019,0.000013,0.000009,0.000007,0.000005,0.000004,0.000002,1.586066e-06,1.076878e-06,8.531343e-07,6.507814e-07,3.157767e-07,2.393200e-07,2.209751e-07,1.868328e-07,1.590003e-07,6.759533e-08,1.000932e-08,7.519310e-09,calm,0.0
1,-599.887329,64.439102,-6.381915,10.303803,-6.244401,2.290793,-12.671721,-7.977162,-14.238932,-14.016172,-9.058470,-3.350154,-9.087075,-11.281478,0.246066,-2.009473,-9.165910,-0.663268,-5.202428,-7.709608,-2.851307,-2.003413,-4.028022,-2.931092,-0.774375,1.246649,-1.521887,0.335499,1.098818,1.912535,0.491459,3.205811,2.845409,3.042243,3.754850,2.933762,1.089847,0.870790,-1.424171,-1.535988,...,0.001785,0.001842,0.002122,0.002454,0.002326,0.001533,0.001448,0.001170,0.001074,0.000942,0.000509,0.000414,0.000397,0.000407,0.000394,0.000367,0.000308,0.000310,0.000297,0.000263,0.000259,0.000196,0.000153,0.000131,0.000146,0.000090,4.610521e-05,2.757442e-05,2.128181e-05,2.384256e-05,2.641989e-05,2.769627e-05,2.345415e-05,1.330578e-05,8.395259e-06,2.400611e-06,1.909306e-07,1.025534e-08,sad,NaN
2,-714.944580,61.081409,-3.209198,6.729263,-4.052040,0.417168,-9.017234,-1.867722,-12.592785,-8.722701,-5.510410,-8.257329,-5.900076,-5.306100,-2.403667,-6.551586,-5.104366,1.111306,-7.570882,-5.997841,-3.218534,-3.474436,-3.770219,-5.410140,-4.613837,-3.946187,-4.912663,-3.068431,-3.998256,-1.903009,-0.331806,-1.932699,-1.248906,0.527815,0.530568,0.129738,0.377466,1.127840,1.971075,3.363005,...,0.000034,0.000037,0.000032,0.000034,0.000024,0.000013,0.000009,0.000006,0.000005,0.000003,0.000003,0.000003,0.000004,0.000006,0.000007,0.000010,0.000013,0.000013,0.000008,0.000005,0.000009,0.000006,0.000003,0.000002,0.000002,0.000001,8.747767e-07,4.782062e-07,3.536885e-07,2.087800e-07,1.781192e-07,1.589432e-07,1.324980e-07,9.580431e-08,5.103842e-08,2.411450e-08,7.413765e-09,5.675293e-09,sad,NaN
3,-560.943909,59.007679,-21.721348,4.099717,-3.345491,-0.744794,-19.210684,-1.305787,-7.590088,-8.393338,-2.182273,-3.921418,-7.888694,-2.158708,-0.176220,-6.954059,-4.602485,3.350716,-7.865934,-4.916510,-2.450394,-3.382827,-4.303576,-0.807909,-1.455294,-0.107363,-1.840241,3.126824,1.385932,2.728318,2.840037,1.761733,0.383645,1.888118,2.619172,2.096194,5.148524,5.815701,3.486905,2.302528,...,0.000374,0.000408,0.000405,0.000266,0.000247,0.000241,0.000162,0.000195,0.000212,0.000175,0.000134,0.000136,0.000145,0.000192,0.000242,0.000223,0.000234,0.000215,0.000211,0.000125,0.000111,0.000074,0.000054,0.000042,0.000028,0.000020,1.370840e-05,9.621894e-06,1.033495e-05,7.868209e-06,6.618394e-06,5.796551e-06,3.801436e-06,2.812426e-06,2.378993e-06,9.983867e-07,8.456907e-08,8.641630e-09,happy,1.0
4,-549.236206,55.645756,-18.499104,3.886537,2.170367,-1.320795,-13.935467,-1.307494,-6.164294,-6.724278,-1.748288,-2.759629,-6.336930,-2.210993,3.369204,-3.728796,-4.467932,2.997232,-4.070324,-4.058586,-1.952857,-2.550913,-3.447933,-1.292407,-1.157766,-1.425361,-2.301708,0.266337,0.086388,-0.239441,0.076293,1.605086,1.152786,3.166218,2.452166,-0.048638,2.362867,3.411358,4.714571,6.882650,...,0.000492,0.000452,0.000297,0.000519,0.000333,0.000296,0.000229,0.000229,0.000251,0.000127,0.000074,0.000065,0.000095,0.000208,0.000307

In [ ]:
df.drop('label', axis=1, inplace=True)

In [ ]:
df.to_csv('data_reduced.csv')

In [ ]:
df = df.fillna(lambda x: x.median())

In [ ]:
new=pd.read_csv('data_reduced.csv')
new=new.iloc[:,1:]

print(new.shape)
new.head()

(968, 181)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,num_label
0,-712.388306,63.240341,-6.698063,12.928182,-9.692841,7.185095,-12.298142,-2.932858,-8.628733,-5.958882,-8.213329,-8.318868,-3.540151,-5.391190,-5.856447,-5.400100,-6.518381,-0.311035,-7.376580,-4.583142,-4.758417,-5.163467,-4.300497,-4.019840,-5.225996,-2.069189,-5.919728,-3.028947,-3.619289,-1.668529,-1.200602,-0.838987,-0.979882,2.792879,1.156514,1.210668,0.850648,1.873484,1.920929,5.635485,...,0.000432,0.000264,0.000327,0.000348,0.000256,0.000199,0.000112,0.000069,0.000049,0.000049,0.000051,0.000041,0.000027,0.000026,0.000041,0.000043,0.000048,0.000035,0.000024,0.000028,0.000019,0.000013,0.000009,0.000007,0.000005,0.000004,0.000002,1.586066e-06,1.076878e-06,8.531343e-07,6.507814e-07,3.157767e-07,2.393200e-07,2.209751e-07,1.868328e-07,1.590003e-07,6.759533e-08,1.000932e-08,7.519310e-09,0.0
1,-599.887329,64.439102,-6.381915,10.303803,-6.244401,2.290793,-12.671721,-7.977162,-14.238932,-14.016172,-9.058470,-3.350154,-9.087075,-11.281478,0.246066,-2.009473,-9.165910,-0.663268,-5.202428,-7.709608,-2.851307,-2.003413,-4.028022,-2.931092,-0.774375,1.246649,-1.521887,0.335499,1.098818,1.912535,0.491459,3.205811,2.845409,3.042243,3.754850,2.933762,1.089847,0.870790,-1.424171,-1.535988,...,0.002415,0.001785,0.001842,0.002122,0.002454,0.002326,0.001533,0.001448,0.001170,0.001074,0.000942,0.000509,0.000414,0.000397,0.000407,0.000394,0.000367,0.000308,0.000310,0.000297,0.000263,0.000259,0.000196,0.000153,0.000131,0.000146,0.000090,4.610521e-05,2.757442e-05,2.128181e-05,2.384256e-05,2.641989e-05,2.769627e-05,2.345415e-05,1.330578e-05,8.395259e-06,2.400611e-06,1.909306e-07,1.025534e-08,NaN
2,-714.944580,61.081409,-3.209198,6.729263,-4.052040,0.417168,-9.017234,-1.867722,-12.592785,-8.722701,-5.510410,-8.257329,-5.900076,-5.306100,-2.403667,-6.551586,-5.104366,1.111306,-7.570882,-5.997841,-3.218534,-3.474436,-3.770219,-5.410140,-4.613837,-3.946187,-4.912663,-3.068431,-3.998256,-1.903009,-0.331806,-1.932699,-1.248906,0.527815,0.530568,0.129738,0.377466,1.127840,1.971075,3.363005,...,0.000036,0.000034,0.000037,0.000032,0.000034,0.000024,0.000013,0.000009,0.000006,0.000005,0.000003,0.000003,0.000003,0.000004,0.000006,0.000007,0.000010,0.000013,0.000013,0.000008,0.000005,0.000009,0.000006,0.000003,0.000002,0.000002,0.000001,8.747767e-07,4.782062e-07,3.536885e-07,2.087800e-07,1.781192e-07,1.589432e-07,1.324980e-07,9.580431e-08,5.103842e-08,2.411450e-08,7.413765e-09,5.675293e-09,NaN
3,-560.943909,59.007679,-21.721348,4.099717,-3.345491,-0.744794,-19.210684,-1.305787,-7.590088,-8.393338,-2.182273,-3.921418,-7.888694,-2.158708,-0.176220,-6.954059,-4.602485,3.350716,-7.865934,-4.916510,-2.450394,-3.382827,-4.303576,-0.807909,-1.455294,-0.107363,-1.840241,3.126824,1.385932,2.728318,2.840037,1.761733,0.383645,1.888118,2.619172,2.096194,5.148524,5.815701,3.486905,2.302528,...,0.000360,0.000374,0.000408,0.000405,0.000266,0.000247,0.000241,0.000162,0.000195,0.000212,0.000175,0.000134,0.000136,0.000145,0.000192,0.000242,0.000223,0.000234,0.000215,0.000211,0.000125,0.000111,0.000074,0.000054,0.000042,0.000028,0.000020,1.370840e-05,9.621894e-06,1.033495e-05,7.868209e-06,6.618394e-06,5.796551e-06,3.801436e-06,2.812426e-06,2.378993e-06,9.983867e-07,8.456907e-08,8.641630e-09,1.0
4,-549.236206,55.645756,-18.499104,3.886537,2.170367,-1.320795,-13.935467,-1.307494,-6.164294,-6.724278,-1.748288,-2.759629,-6.336930,-2.210993,3.369204,-3.728796,-4.467932,2.997232,-4.070324,-4.058586,-1.952857,-2.550913,-3.447933,-1.292407,-1.157766,-1.425361,-2.301708,0.266337,0.086388,-0.239441,0.076293,1.605086,1.152786,3.166218,2.452166,-0.048638,2.362867,3.411358,4.714571,6.882650,...,0.000423,0.000492,0.000452,0.000297,0.000519,0.000333,0.000296,0.000229,0.000229,0.000251,0.000127,0.000074,0.000065,0.

In [ ]:
new['num_label'].value_counts()

2.0    199
1.0    191
0.0    190
Name: num_label, dtype: int64

In [ ]:
#x_train,x_test,y_train,y_test=train_test_split(new.iloc[:,:-1],new['num_label'], test_size=0.25, random_state=9)
#x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
#x_train.shape
print((x_train.shape[0], x_test.shape[0]))

(726, 242)


In [ ]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 62.40%
